In [1]:
import pandas as pd

# Load data
projects = pd.read_csv('C:/Users/zarin/OneDrive/Рабочий стол/ML data/projects.csv')
donations = pd.read_csv('C:/Users/zarin/OneDrive/Рабочий стол/ML data/donations.csv')
outcomes = pd.read_csv('C:/Users/zarin/OneDrive/Рабочий стол/ML data/outcomes.csv')
resources = pd.read_csv('C:/Users/zarin/OneDrive/Рабочий стол/ML data/resources.csv')


In [4]:
import pandas as pd
import numpy as np

df = projects.merge(outcomes[['projectid', 'fully_funded']], on='projectid', how='inner')




In [6]:
#Feature Engineering
resource_agg = resources.groupby('projectid').agg({
    'item_unit_price': ['mean', 'max', 'min'],
    'item_quantity': ['sum'],
    'item_name': 'nunique'
}).reset_index()

resource_agg.columns = [
    'projectid',
    'avg_unit_price', 'max_unit_price', 'min_unit_price',
    'total_quantity', 'unique_items'
]


In [7]:

donation_agg = donations.groupby('projectid').agg({
    'donation_to_project': ['sum', 'count', 'mean'],
    'is_teacher_acct': 'sum',
    'payment_was_promo_matched': 'sum'
}).reset_index()

donation_agg.columns = [
    'projectid',
    'total_donation_amt', 'donation_count', 'avg_donation',
    'num_teacher_donations', 'num_promo_matched'
]

In [8]:
#  Merge everything into main DataFrame
df = df.merge(resource_agg, on='projectid', how='left') \
       .merge(donation_agg, on='projectid', how='left')

In [9]:
df['date_posted'] = pd.to_datetime(df['date_posted'], errors='coerce')
df['posted_month'] = df['date_posted'].dt.month
df['posted_dayofweek'] = df['date_posted'].dt.dayofweek


In [10]:

categorical_cols = [
    'school_metro', 'school_state', 'poverty_level', 'grade_level',
    'primary_focus_area', 'primary_focus_subject',
    'resource_type', 'school_charter', 'school_magnet'
]

df[categorical_cols] = df[categorical_cols].fillna('Unknown')

df_encoded = pd.get_dummies(df[categorical_cols], drop_first=True)

In [11]:

numeric_cols = [
    'students_reached',
    'total_price_excluding_optional_support',
    'total_price_including_optional_support',
    'avg_unit_price', 'max_unit_price', 'min_unit_price',
    'total_quantity', 'unique_items',
    'total_donation_amt', 'donation_count', 'avg_donation',
    'num_teacher_donations', 'num_promo_matched',
    'posted_month', 'posted_dayofweek'
]

df[numeric_cols] = df[numeric_cols].fillna(0)

X = pd.concat([df[numeric_cols], df_encoded], axis=1)
y = df['fully_funded']

# Final feature matrix ready for modeling!
print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")

Feature matrix shape: (619326, 118)
Target vector shape: (619326,)


In [14]:
feature_matrix = X.copy()
feature_matrix['fully_funded'] = y

feature_matrix.to_csv('feature_matrix.csv', index=False)


In [13]:
import os
print(os.getcwd())


C:\Users\zarin
